#### The work of this file is to generate the adversarial sample of the test data.

##### Training the defense model.

In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals
import logging
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, Activation, Dropout
import numpy as np
from art.attacks.evasion import DeepFool
from art.estimators.classification import KerasClassifier
from art.utils import load_dataset
import tensorflow as tf
tf.compat.v1.disable_eager_execution()
from art.attacks.evasion import FastGradientMethod, BasicIterativeMethod, CarliniL2Method, CarliniLInfMethod, ProjectedGradientDescent, DeepFool, ThresholdAttack, PixelAttack, SpatialTransformation, SquareAttack, ZooAttack, BoundaryAttack, HopSkipJump

(x_train, y_train), (x_test, y_test), min_, max_ = load_dataset(str("mnist"))
x_train, y_train = x_train[:5000], y_train[:5000]
x_test, y_test = x_test[:500], y_test[:500]
im_shape = x_train[0].shape


# The structure of defense model.
model = Sequential()
model.add(Conv2D(32, (5, 5), input_shape= x_train.shape[1:], activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(10, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

classifier = KerasClassifier(model=model, clip_values=(min_, max_))
classifier.fit(x_train, y_train, nb_epochs=10, batch_size=128)

Train on 5000 samples
Epoch 1/10
5000/5000 [==============================] - 2s 466us/sample - loss: 0.8070 - accuracy: 0.77320s - loss: 0.8551 - accura
Epoch 2/10
5000/5000 [==============================] - 2s 440us/sample - loss: 0.2757 - accuracy: 0.9222
Epoch 3/10
5000/5000 [==============================] - 2s 435us/sample - loss: 0.1824 - accuracy: 0.9480
Epoch 4/10
5000/5000 [==============================] - 2s 449us/sample - loss: 0.1398 - accuracy: 0.9588 - loss: 0 - ETA: 1s - loss: 0.1404 - accuracy
Epoch 5/10
5000/5000 [==============================] - 2s 427us/sample - loss: 0.1083 - accuracy: 0.9686
Epoch 6/10
5000/5000 [==============================] - 2s 420us/sample - loss: 0.0857 - accuracy: 0.9762
Epoch 7/10
5000/5000 [==============================] - 2s 435us/sample - loss: 0.0766 - accuracy: 0.9792
Epoch 8/10
5000/5000 [==============================] - 2s 435us/sample - loss: 0.0587 - accuracy: 0.9848 - loss:
Epoch 9/10
5000/5000 [============================

##### Building the generator of the adversarial example.

In [2]:
from art.attacks.evasion import CarliniL2Method, SaliencyMapMethod, FastGradientMethod, ProjectedGradientDescent
crafter_cl = CarliniL2Method(classifier=classifier, max_iter=200) #CW2
crafter_jsma = SaliencyMapMethod(classifier=classifier) #JSMA
crafter_fgsm = FastGradientMethod(classifier) # eps = 0.3, bath_size = 32 #FGSM
crafter_pgd = ProjectedGradientDescent(estimator=classifier, norm=np.inf, eps=0.3, eps_step=0.01, batch_size=32) #PGD

##### Generating the adversarial example and measuring the time delay of the generation of the adversarial example.

In [ ]:
import time
# CW2
time_start_cw2 = time.time()
x_test_adv_cw2 = crafter_cl.generate(x_test)
time_end_cw2 = time.time()
average_time_cw2 = (time_end_cw2 - time_start_cw2)/500
#JSMA
A = np.zeros([500, 10])
def one_hot(A, k):
    A[:,k] = 1
    return A
for i in range(0,500,50):
    one_hot(A[i:i+50,:],int(i/50))
time_start_jsma = time.time()
x_test_adv_jsma = crafter_jsma.generate(x_test, A)
time_end_jsma = time.time()
average_time_jsma = (time_end_jsma - time_start_jsma)/500
#FGSM
time_start_fgsm = time.time()
x_test_adv_fgsm = crafter_fgsm.generate(x_test)
time_end_fgsm = time.time()
average_time_fgsm = (time_end_fgsm - time_start_fgsm)/500
#PGD
time_start_pgd = time.time()
x_test_adv_pgd = crafter_pgd.generate(x_test)
time_end_pgd = time.time()
average_time_pgd = (time_end_pgd - time_start_pgd)/500
#Save the adversarial example of the test data.
np.save("./x_test_adv_cw2.npy", x_test_adv_cw2)
np.save("./x_test_adv_jsma.npy", x_test_adv_jsma)
np.save("./x_test_adv_fgsm.npy", x_test_adv_fgsm)
np.save("./x_test_adv_pgd.npy", x_test_adv_pgd)
#Print the time delay generating the adversarial examples for different methods.
print(average_time_cw2)
print(average_time_jsma)
print(average_time_fgsm)
print(average_time_pgd)

C&W L_2:   0%|          | 0/500 [00:00<?, ?it/s]